In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt

In [2]:
dis_acct_e = pd.concat(pd.read_csv('dis_acct_e.csv', chunksize=1000, date_parser=pd.datetools.to_datetime))

In [3]:
dis_acct_e.head()

,me_dt,acct_id,acct_sub_type,acct_opnd_dt,acct_opng_brnch_id,acct_sts,cust_id,prod_id,fincg_type_cd,tenor,...,inlien_ind,cury_cd,bal_outst_amt,avg_bal_out_globl_amt,avg_bal_out_globl_amt_cr,avg_bal_out_globl_amt_dr,fund_cr_amt,fund_chrg_amt,inlien_amt,no_of_time_renewed
0,2015-12-31,18124905,CURRENT ACCOUNT/OVERDRAFT,2013-07-18,3426827,CLOSE,11716809,6302,191,NaN,...,N,MYR,16.289508,15.649508,15.969508,24.839508,15.679508,7.129508,16.259508,0
1,2015-12-31,17816466,CURRENT ACCOUNT/OVERDRAFT,2013-04-10,3426532,CLOSE,11503652,6302,191,NaN,...,N,MYR,73659.291778,44321.011778,58990.151778,78314.921778,44385.081778,39729.451778,73595.221778,0
2,2015-12-31,2189400,CURRENT ACCOUNT/OVERDRAFT,2013-11-27,3426268,ACTIVE,116097,5831,111,NaN,...,N,MYR,172067.371356,84835.531356,128451.451356,158898.531356,84934.011356,98102.851356,171968.891356,0
3,2015-12-31,17851353,CURRENT ACCOUNT/OVERDRAFT,2014-07-20,3426565,ACTIVE,12568229,6302,191,NaN,...,N,MYR,1969.166978,1094.726978,1531.946978,1977.716978,1096.206978,1087.656978,1967.686978,0
4,2015-12-31,18208198,CURRENT ACCOUNT/OVERDRAFT,2014-01-22,3426892,DORMANT,12123857,6302,191,NaN,...,N,MYR,44622.457314,24656.457314,34639.457314,44622.457314,24689.557314,24689.557314,44589.357314,0


In [4]:
# dis_acct_e['me_dt'] = dis_acct_e['me_dt'].astype('datetime64[ns]')
dis_acct_e['me_dt'] = pd.to_datetime(dis_acct_e['me_dt'])
dis_acct_e['acct_opnd_dt'] = pd.to_datetime(dis_acct_e['acct_opnd_dt'])
dis_acct_e['cert_iss_dt'] = pd.to_datetime(dis_acct_e['cert_iss_dt'])
dis_acct_e['dep_maturity_dt'] = pd.to_datetime(dis_acct_e['dep_maturity_dt'])

print(dis_acct_e['me_dt'].dtype)

datetime64[ns]


In [5]:
dis_acct_e_nonInt = dis_acct_e.set_index('acct_id').select_dtypes(exclude=['floating', 'int', 'datetime64[ns]'])
dis_acct_e_nonInt.head()

,acct_sub_type,acct_sts,tenor_uom,remain_term_uom,maturity_handle_codes,joint_acct_ind,new_acct_ind,inlien_ind,cury_cd
acct_id,,,,,,,,,
18124905,CURRENT ACCOUNT/OVERDRAFT,CLOSE,NaN,NaN,NaN,N,N,N,MYR
17816466,CURRENT ACCOUNT/OVERDRAFT,CLOSE,NaN,NaN,NaN,N,N,N,MYR
2189400,CURRENT ACCOUNT/OVERDRAFT,ACTIVE,NaN,NaN,NaN,N,N,N,MYR
17851353,CURRENT ACCOUNT/OVERDRAFT,ACTIVE,NaN,NaN,NaN,N,N,N,MYR
18208198,CURRENT ACCOUNT/OVERDRAFT,DORMANT,NaN,NaN,NaN,N,N,N,MYR


In [6]:
dis_acct_e_nonInt.iloc[:,0:].apply(pd.Series.value_counts)

,acct_sub_type,acct_sts,tenor_uom,remain_term_uom,maturity_handle_codes,joint_acct_ind,new_acct_ind,inlien_ind,cury_cd
ACTIVE,NaN,1008399.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AUTO RENEW,NaN,NaN,NaN,NaN,59541.0,NaN,NaN,NaN,NaN
CLOSE,NaN,100953.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CURRENT ACCOUNT/OVERDRAFT,1006236.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DORMANT,NaN,35042.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FIXED DEPOSIT - CERTIFICATE,138158.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MONTHS,NaN,NaN,138158.0,138158.0,NaN,NaN,NaN,NaN,NaN
MYR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1144394.0
N,NaN,NaN,NaN,NaN,NaN,1144274.0,1128424.0,1126376.0,NaN
NO AUTO RENEW,NaN,NaN,NaN,NaN,529.0,NaN,NaN,NaN,NaN


In [7]:
dis_acct_e_nonInt_vec = pd.get_dummies(dis_acct_e_nonInt)

In [8]:
print(len(dis_acct_e_nonInt_vec))
dis_acct_e_nonInt_vec.head()

1144394


,acct_sub_type_CURRENT ACCOUNT/OVERDRAFT,acct_sub_type_FIXED DEPOSIT - CERTIFICATE,acct_sts_ACTIVE,acct_sts_CLOSE,acct_sts_DORMANT,tenor_uom_MONTHS,remain_term_uom_MONTHS,maturity_handle_codes_AUTO RENEW,maturity_handle_codes_NO AUTO RENEW,joint_acct_ind_N,joint_acct_ind_Y,new_acct_ind_N,new_acct_ind_Y,inlien_ind_N,inlien_ind_Y,cury_cd_MYR
acct_id,,,,,,,,,,,,,,,,
18124905,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
17816466,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
2189400,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
17851353,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
18208198,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0


In [9]:
print(dis_acct_e.columns.difference(dis_acct_e_nonInt.columns))
dis_acct_e_int1 = dis_acct_e.ix[:, dis_acct_e.columns.difference(dis_acct_e_nonInt.columns)]
dis_acct_e_int1.head()

Index([u'acct_id', u'acct_opnd_dt', u'acct_opng_brnch_id',
       u'avg_bal_out_globl_amt', u'avg_bal_out_globl_amt_cr',
       u'avg_bal_out_globl_amt_dr', u'bal_outst_amt', u'cert_iss_dt',
       u'cust_id', u'dep_maturity_dt', u'fincg_type_cd', u'fund_chrg_amt',
       u'fund_cr_amt', u'inlien_amt', u'me_dt', u'no_of_time_renewed',
       u'prod_id', u'remain_term', u'tenor'],
      dtype='object')


,acct_id,acct_opnd_dt,acct_opng_brnch_id,avg_bal_out_globl_amt,avg_bal_out_globl_amt_cr,avg_bal_out_globl_amt_dr,bal_outst_amt,cert_iss_dt,cust_id,dep_maturity_dt,fincg_type_cd,fund_chrg_amt,fund_cr_amt,inlien_amt,me_dt,no_of_time_renewed,prod_id,remain_term,tenor
0,18124905,2013-07-18,3426827,15.649508,15.969508,24.839508,16.289508,NaT,11716809,NaT,191,7.129508,15.679508,16.259508,2015-12-31,0,6302,0,NaN
1,17816466,2013-04-10,3426532,44321.011778,58990.151778,78314.921778,73659.291778,NaT,11503652,NaT,191,39729.451778,44385.081778,73595.221778,2015-12-31,0,6302,0,NaN
2,2189400,2013-11-27,3426268,84835.531356,128451.451356,158898.531356,172067.371356,NaT,116097,NaT,111,98102.851356,84934.011356,171968.891356,2015-12-31,0,5831,0,NaN
3,17851353,2014-07-20,3426565,1094.726978,1531.946978,1977.716978,1969.166978,NaT,12568229,NaT,191,1087.656978,1096.206978,1967.686978,2015-12-31,0,6302,0,NaN
4,18208198,2014-01-22,3426892,24656.457314,34639.457314,44622.457314,44622.457314,NaT,12123857,NaT,191,24689.557314,24689.557314,44589.357314,2015-12-31,0,6302,0,NaN


In [10]:
dis_acct_e_int2 = dis_acct_e_int1.set_index('acct_id')
# dis_acct_e_nonInt_vec.head()

In [25]:
# pd.merge(dis_acct_e_int2, dis_acct_e_nonInt_vec, on=dis_acct_e_int2.index)
dis_acct_e_vectorized = dis_acct_e_int2.join(dis_acct_e_nonInt_vec)

In [26]:
dis_acct_e_vectorized.columns

Index([u'acct_opnd_dt', u'acct_opng_brnch_id', u'avg_bal_out_globl_amt',
       u'avg_bal_out_globl_amt_cr', u'avg_bal_out_globl_amt_dr',
       u'bal_outst_amt', u'cert_iss_dt', u'cust_id', u'dep_maturity_dt',
       u'fincg_type_cd', u'fund_chrg_amt', u'fund_cr_amt', u'inlien_amt',
       u'me_dt', u'no_of_time_renewed', u'prod_id', u'remain_term', u'tenor',
       u'acct_sub_type_CURRENT ACCOUNT/OVERDRAFT',
       u'acct_sub_type_FIXED DEPOSIT - CERTIFICATE', u'acct_sts_ACTIVE',
       u'acct_sts_CLOSE', u'acct_sts_DORMANT', u'tenor_uom_MONTHS',
       u'remain_term_uom_MONTHS', u'maturity_handle_codes_AUTO RENEW',
       u'maturity_handle_codes_NO AUTO RENEW', u'joint_acct_ind_N',
       u'joint_acct_ind_Y', u'new_acct_ind_N', u'new_acct_ind_Y',
       u'inlien_ind_N', u'inlien_ind_Y', u'cury_cd_MYR'],
      dtype='object')